In [3]:
import os
import pinocchio as pin
import numpy as np
import matplotlib.pyplot as plt

# pin.initGnuplot()

# set default print options
np.set_printoptions(precision=4, suppress=True)

# create robot model using pinocchio
urdf_path = '/home/helei/catkin_eagle_mpc/src/eagle_mpc_debugger/models/urdf/s500_uam_simple.urdf'
model = pin.buildModelFromUrdf(urdf_path, pin.JointModelFreeFlyer())
data = model.createData()

state_dim = model.nq + model.nv  # number of state using quaternion
state_dim_euler = state_dim - 1                   # number of state using euler angle
control_dim = model.nv
arm_joint_num = model.nq - 7

print(f"state_dim: {state_dim}")
print(f"state_dim_euler: {state_dim_euler}")
print(f"control_dim: {control_dim}")
print(f"arm_joint_num: {arm_joint_num}")

q0 = pin.neutral(model)               # default “neutral” configuration
v0 = pin.utils.zero(model.nv)         # zero velocity

# get inertia matrix
inertia_matrix = pin.crba(model, data, q0)

# get coriolis matrix
# coriolis_matrix = pin.coriolis(model, data, q0, v0)

# # get gravity vector
# gravity_vector = pin.rnea(model, data, q0, v0, np.zeros(model.nv))

# # get mass matrix
# mass_matrix = pin.mass(model, data, q0)

# # get mass matrix inverse
# mass_matrix_inverse = np.linalg.inv(mass_matrix)
for i in range(8):
    print(inertia_matrix[:, i])  


state_dim: 17
state_dim_euler: 16
control_dim: 8
arm_joint_num: 2
[ 2.014   0.      0.     -0.     -0.0389  0.     -0.027  -0.0091]
[ 0.      2.014   0.      0.0389 -0.      0.0002  0.      0.    ]
[ 0.      0.      2.014  -0.     -0.0002 -0.     -0.     -0.    ]
[-0.      0.0389 -0.      0.0273 -0.      0.     -0.     -0.    ]
[-0.0389 -0.     -0.0002 -0.      0.0296 -0.      0.0064  0.0025]
[ 0.      0.0002 -0.      0.     -0.      0.0276 -0.     -0.    ]
[-0.027   0.     -0.     -0.      0.0064 -0.      0.0044  0.0018]
[-0.0091  0.     -0.     -0.      0.0025 -0.      0.0018  0.0009]


In [ ]:
q1 = q0.copy()

q1[7] = 1.2
q1[8] = 0.6

print(q1)

inertia_matrix = pin.crba(model, data, q1)

for i in range(8):
    print(inertia_matrix[:, i])  

[0.  0.  0.  0.  0.  0.  1.  1.2 0.6]
[ 2.014   0.      0.     -0.     -0.0163  0.     -0.0044  0.002 ]
[ 0.      2.014   0.      0.0163 -0.     -0.0255  0.      0.    ]
[ 0.      0.      2.014  -0.      0.0255 -0.      0.0256  0.0089]
[-0.      0.0163 -0.      0.0198 -0.     -0.0023 -0.     -0.    ]
[-0.0163 -0.      0.0255 -0.      0.0259 -0.      0.0044  0.0015]
[ 0.     -0.0255 -0.     -0.0023 -0.      0.0315 -0.      0.    ]
[-0.0044  0.      0.0256 -0.      0.0044 -0.      0.0041  0.0017]
[ 0.002   0.      0.0089 -0.      0.0015  0.      0.0017  0.0009]


Get relationship between K and dt, As

In [29]:
import numpy as np
import scipy.linalg as sLA

def get_K_matrix(As, dt, state_dim_euler):
    a_s = np.ones(state_dim_euler) * As #! Hurwitz matrix
        
    # Pre-compute matrices for optimization
    A_s = np.diag(a_s)  # diagonal Hurwitz matrix
    expm_A_s_dt = sLA.expm(A_s * dt)
    
    # Pre-compute PHI inverse elements since A_s is diagonal
    # For a diagonal matrix A, exp(A*dt) is a diagonal matrix with exp(a_ii*dt)
    # And (exp(A*dt) - I) is diagonal with (exp(a_ii*dt) - 1)
    # Therefore PHI = A^(-1)(exp(A*dt) - I) is diagonal with (exp(a_ii*dt) - 1)/a_ii
    expm_minus_I = expm_A_s_dt - np.identity(state_dim_euler)
    PHI_diag = expm_minus_I.diagonal() / a_s
    PHI_inv_diag = 1.0 / PHI_diag  # Pre-compute the inverse elements
    
    return PHI_inv_diag

def get_K(As, dt):
    # a_s = np.ones(state_dim_euler) * As #! Hurwitz matrix
        
    expm_A_s_dt = np.exp(As * dt)

    expm_A_s_dt_minus_I = expm_A_s_dt - 1

    Phi = expm_A_s_dt_minus_I / As

    K = expm_A_s_dt / Phi
    
    return K

In [40]:
As = -1
dt = 0.01
state_dim_euler = 8

print(get_K_matrix(As, dt, state_dim_euler))
print(get_K(As, dt))

[100.5008 100.5008 100.5008 100.5008 100.5008 100.5008 100.5008 100.5008]
99.50083333194499
